# The bootstrap filter {#sec-smc-bootstrap}

<!-- ```{julia}
#| echo: false
#| output: false

# Setting up
using Plots, Measures, Distributions
include("../src/loadData.jl")
``` -->


In this chapter we introduce a variant of the **bootstrap filter**.

The bootstrap filter generates sample-based approximations to the conditional filtering distribution $P(X_t | y_{1:t}, \theta)$ and conditional smoothing distribution $P(X_t | y_{1:T}, \theta)$.

*This chapter assumes fixed and known values of the model parameters $\theta$. @sec-smc-parameterestimation handles the estimation of $\theta$.*


## Intuition

The goal of the bootstrap filter is to use observed data $y_{1:T}$ to learn about the hidden-states $X_t$.

::: {.callout-tip}
We provide a different intuitive explanation in the corresponding [journal article](https://github.com/nicsteyn2/SMCforRt/blob/main/smcforRt.pdf). Otherwise see @sec-smc-bootstrap-additional for additional resources.
:::

### Sequential Importance (Re)sampling

Assume that we have a collection of samples from the *filtering distribution* at time $t\!-\!1$, denoted $\{x_{t-1}^{(i)}\} \sim P(X_{t-1}|y_{1:t-1}, \theta)$. We call these samples **particles**, and they collectively represent our knowledge about the hidden-states at this time step.

By coupling these particles with the state-space transition model (@eq-intro-statespace), we can make one-step-ahead predictions. Mathematically we want to obtain the predictive distribution:
$$
\underbrace{P(X_t | y_{1:t-1}, \theta)}_{\text{predictive dist.}} = \int \underbrace{P(X_t | X_{t-1}, \theta)}_{\text{state-space transition dist.}} \underbrace{P(X_{t-1}|y_{1:t-1}, \theta)}_{\text{prev. filtering dist.}} \ dX_{t-1}
$$
Which is a complicated and potentially high-dimensional integral. Fortunately, all we need to do is sample from the state-space transition model while conditioning on our particles:
$$
\{\tilde{x}_t^{(i)}\} \sim P(X_t | x_{t-1}^{(i)}, \theta)
$$

If we treat this predictive distribution as the prior distribution for $X_t|y_{1:t}$, we can apply Bayes' formula:

$$
\underbrace{P(X_t | y_{1:t}, \theta)}_{\text{new filtering dist}} \propto \underbrace{P(y_t|X_t, y_{1:t-1}, \theta)}_{\text{observation dist.}} \underbrace{P(X_t | y_{1:t-1}, \theta)}_{\text{predictive dist.}}
$$

Since we already have samples from $P(X_t | y_{1:t-1}, \theta)$, all we need to do is assign them weights $w_t^{(i)}$ according to the observation distribution:

$$
w_t^{(i)} = P(y_t | \tilde{x}_t^{(i)}, y_{1:t-1}, \theta)
$$

The final set of particles $\{x_t^{(i)}\}$ is constructed by re-sampling (with replacement) from $\{\tilde{x}_t^{(i)}\}$ according to the corresponding weights. Thus:

$$
x_t^{(i)} \sim P(X_t | y_{1:t}, \theta)
$$

Thus, starting with an initial set of particles $\{x_0^{(i)}\} \sim P(X_0)$, all we need to do is repeat this procedue at each time step $t$, storing the particle values as we go. If the resampling-step is performed at every time step where data are observed, this is called a **bootstrap filter**, which is a special case of a sequential importance resampling algorithm.

### State-history resampling

In most 

The derivation provided above assumes that our model is Markovian: the state-space transition and observation distributions depend only on $X_{t-1}$, and not on $X_{t-2}, X_{t-3}, \ldots$.



### Using the particles

## Algorithm

\#TODO: write SMC algorithm

<!--
## Example {#sec-smc-bootstrapexample}

For demonstration, we consider a simple reproduction number estimator. First assume that $\log R_t$ follows a Gaussian random walk:

$$\log R_t \sim \text{Normal}(\log R_{t-1}, \sigma) $$

while reported cases are assumed to follow the Poisson renewal model:

$$ C_t \sim \text{Poisson}\left(R_t \sum_{u=1}^{t-1} C_{t-u} g_u\right) $$

The first equation defines our *hidden-state model* while the second equation defines our *observation model*. With only a slight difference[^1], this model is almost identical to that employed by EpiFilter [@paragImprovedEstimationTimevarying2021].

[^1]: @paragImprovedEstimationTimevarying2021 assumes $R_t$ (rather than $\log R_t$) follows a Gaussian random walk. The standard deviation of this random walk is multiplied by $\sqrt{R_t}$ to allow $R_t$ to take larger "jumps" when it is larger, achieving the same outcome as our log-model. We discuss this further in @sec-other-epifilter.

Leaving parameter estimation to @sec-smc-parameterestimation, we use the following defaults:

In [ ]:
#| output: false

# Serial interval
ω = pdf.(Gamma(2.36, 2.74), 1:100)
ω = ω/sum(ω)

# Smoothing parameter
σ = 0.15

# Initial distribution for Rt
pR0 = Uniform(0, 10) 

Collectively, $\sigma$, $\{\omega_u\}_{u=1}^{u_{max}}$, and $P(R_0)$ constitute the model parameters $\theta$.


### Data {#sec-smc-data}

We use data from the first 100 days of the COVID-19 pandemic in New Zealand. Focusing now on total cases (we leave the critical separation of imported and local cases to @sec-models-imported):


In [ ]:
#| fig-cap: Reported cases from the first 100 days of the COVID-19 pandemic in Aotearoa New Zealand.
#| code-fold: true
#| label: fig-models-nzdata

nzdata = loadData("NZCOVID")
T = length(nzdata.Ct)
bar(nzdata.date, nzdata.Ct, label=false, xlabel="Date", ylabel="Reported cases", size=(800,300), margins=3mm, color=:darkblue)

### Setting up

We need to specify the number of particles $N$ and resampling window $L$:


In [ ]:
#| output: false
N = 10000
L = 50

pre-allocate memory for our particles:


In [ ]:
#| output: false
X = zeros(N, T)

and sample from the initial distribution:


In [ ]:
#| output: false
X[:,1] = rand(pR0, N)

### Implementation

All that's left to do is run the bootstrap filter:


In [ ]:
#| output: false

for tt = 2:T

    # Project according to the state-space model
    X[:,tt] = exp.(rand.(Normal.(log.(X[:,tt-1]), σ)))

    # Weight according to the observation model
    Λ = sum(nzdata.Ct[tt-1:-1:1] .* ω[1:tt-1])
    W = pdf.(Poisson.(X[:,tt] .* Λ), nzdata.Ct[tt])

    # Resample
    inds = wsample(1:N, W, N; replace=true)
    X[:, max(tt - L, 1):tt] = X[inds, max(tt - L, 1):tt]

end

### Results

The $t^{th}$ column of $X$ is a set of samples from $P(X_t | C_{1:t+L})$. The mean and quantiles of this posterior distribution are found using:


In [ ]:
#| fig-cap: Estimated $R_t$ for the first 100 days of the COVID-19 pandemic. The dashed horizontal line indicates $R_t = 1$. The green line shows the posterior mean and the green shading shows the 95% credible interval.
#| label: fig-models-simpleconditionalsmooth

m = [mean(X[:,tt]) for tt in 1:T]
l = [quantile(X[:,tt], 0.025) for tt in 1:T]
u = [quantile(X[:,tt], 0.975) for tt in 1:T]
plot(m, ribbon=(m-l, u-m), color=:darkgreen, label=false, xlabel="Date", ylabel="Reproduction number", size=(800,300), margins=3mm)
hline!([1], label=false, color=:black, line=:dash)

-->

## Resources from other fields {#sec-smc-bootstrap-additional}

Bootstrap filters (and SMC methods more generally) have found use in many fields. Each field has their own motivation for and notation describing these methods. We provide an overview of other resources here.

**Bayesian Filtering and Smoothing [@sarkkaBayesianFilteringSmoothing2013]**

Those with an **engineering background** may be familiar with "filtering and smoothing", where the state of a time-varying system is tracked through the observation of noisy measurements. Classical examples include GPS position tracking or audio signal processing.

The Kalman filter, which provides an analytical solution when the state-space transition and observation models are linear Gaussian and Markovian, is perhaps the best-known example of a filtering method from engineering.

Chapters 7 and 11 of @sarkkaBayesianFilteringSmoothing2013 introduce SMC methods under the headings "particle filtering" and "particle smoothing". We also recommend chapters 1 (*What are Bayesian filtering and smoothing?*), 4 (*Bayesian filtering equations and exact solutions*), and 8 (*Bayesian smoothing equations and exact solutions*).

**A survey of Sequential Monte Carlo Methods for Economics and Finance [@crealSurveySequentialMonte2012]**

Those with an **econometrics background** may find this extensive review helpful, although the author focusses on Markovian models. Examples employed in this review include a stochastic volatility model and a nonlinear dynamic stochastic general equilibrium model.

*This list is incomplete. If you know of any additional resources that may be helpful, [please get in touch](mailto:nicholas.steyn@univ.ox.ac.uk)!*

**Data Assimilation Fundamentals: ... [@evensenDataAssimilationFundamentals2022]**

Those with a background in **atmospheric science, oceanography, metereology, or other environmental sciences** may be familiar with "data assimilation", where focus is placed on combining model predictions with observational data. Chapter 9 of this book introduces particle filters as a method for solving the "fully nonlinear data assimilation" problem.